# 데이터 사전처리(Preprocessing)
- 머신러닝 등 데이터 분석의 정확도는 분석 데이터의 품질에 좌우된다.
- 데이터의 품질을 높이기 위해서는 누락 데이터, 중복 데이터 등 오류를 수정하고 분석목적에 맞게 변형하는 과정이 필요하다
- 수집한 데이터를 분석 목적에 적합하도록 하는 작업을 사전 처리(Preprocessing)이라고 한다

### 누락 데이터(NaN : Not a Number)
- 데이터를 파일로 입력할 때 빠트리거나 파일 형식을 변환하는 과정에서 데이터가 손실되는 경우 발생
- 머신러닝 분석 모형에 데이터를 입력하기 전에 반드시 누락 데이터를 제거하고 다른 적절한 값으로 대체해야 한다.
- 누락 데이터가 많아지면 데이터의 품질이 떨어지고 머신러닝 분석 알고리즘을 왜곡하는 현상이 발생한다.

In [180]:
import seaborn as sns

In [181]:
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


> deck에는 203개의 유요한 범주형 데이터가 있고, 나머지(688개)가 NaN이다.

In [182]:
# deck column의 값 별로 카운트하기, NaN 포함
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

### 누락 데이터 찾기

In [183]:
# head의 NaN 찾기
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [184]:
# head의 NaN이 아닌 값 찾기
df.head().notnull()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [185]:
# 누락 데이터 갯수 확인
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [186]:
# 누락 데이터 제거
missing_df = df.isnull()
missing_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [187]:
# NaN값이 500개 이상인 열을 모두 삭제
def_thresh = df.dropna(thresh=500, axis=1)
def_thresh.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [188]:
# age 열에 나이 데이터가 없는 모든 행 삭제
def_thresh_age = def_thresh.dropna(subset=['age'], how='any', axis=0)
len(def_thresh_age)

714

---
### 누락 데이터의 저장
- 누락 데이터를 무작정 삭제해 버리면 어렵게 수집한 데이터를 활용하지 못한다
- 데이터 중 일부가 누락되어 있더라도 나머지 데이터를 최대한 살려서 활용하는 것이 좋다

In [189]:
df = sns.load_dataset('titanic')

In [190]:
# age 열의 NaN 값을 평균으로 대체
df['age'].fillna(round(df['age'].mean(),1), inplace=True)
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [191]:
# embark_town의 825:829 row 출력 > NaN이 있음
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object

In [196]:
# embark_town의 NaN 값을 최빈값으로 대체
df['embark_town'].fillna(df['embark_town'].value_counts().idxmax(), inplace=True)
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

### 전후 데이터로 값을 치환

In [195]:
df = sns.load_dataset('titanic')

# embark_town의 NaN 값을 앞의 값으로 대체
df['embark_town'].fillna(method='rfill', inplace=True) # bfill, ffill 사용 가능
df['embark_town'][825:830]

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got rfill

---
# 중복 데이터 처리
- 동일한 대상이 중복으로 존재하는 것은 분석 결과의 왜곡을 초래한다

In [197]:
import pandas as pd

In [198]:
df = pd.DataFrame({'c1': ['a', 'a', 'b', 'a', 'b'], 'c2': [1, 1, 1, 2, 2], 'c3': [1, 1, 2, 2, 2]})
df

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [199]:
# df에서 중복된 row 삭제 (col을 찾을 수는 없음)
df.drop_duplicates(inplace=True)
df

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


In [ ]:
# 특정 열에서 중복된 데이터 찾기
df.duplicated(subset=['c2'])
df['c2'].duplicated()

0    False
2     True
3    False
4     True
Name: c2, dtype: bool

In [ ]:
df = pd.DataFrame({'c1': ['a', 'a', 'b', 'a', 'b'], 'c2': [1, 1, 1, 2, 2], 'c3': [1, 1, 2, 2, 2]})

# c2와 c3열을 기준으로 중복된 row 삭제
df.drop_duplicates(['c2', 'c3'], inplace=True)
# df[['c2', 'c3']].drop_duplicates(inplace=True) # 위와 같음
df

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


---
# 데이터 표준화
- 여러곳에서 수집한 데이터는 서로 다른 단위가 섞여있거나, 대상을 다른 형식으로 표현하는 경우가 많다
- 대상을 표현하는 방법에 차이가 있으면 분석의 정확도는 현저히 낮아진다
- 데이터 포맷을 일관성 있게 하는 작업이 표준화이다. 

In [201]:
df = pd.read_csv('data/auto-mpg.csv')
# mpg to kpl
df['kpl'] = round(df['mpg'] * 1.60934 / 3.78541, 2)

df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.23


### 자료형 변환

In [ ]:
# 각 열의 값의 타입을 출력
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
 9   kpl           398 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 31.2+ KB


In [202]:
# horsepower열의 고유값을 출력
df['horsepower'].unique()

# '?' 값 찾아내기
mask = df['horsepower'] == '?' 
df[mask]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
32,25.0,4,98.0,?,2046.0,19.0,71,1,ford pinto,10.63
126,21.0,6,200.0,?,2875.0,17.0,74,1,ford maverick,8.93
330,40.9,4,85.0,?,1835.0,17.3,80,2,renault lecar deluxe,17.39
336,23.6,4,140.0,?,2905.0,14.3,80,1,ford mustang cobra,10.03
354,34.5,4,100.0,?,2320.0,15.8,81,2,renault 18i,14.67
374,23.0,4,151.0,?,3035.0,20.5,82,1,amc concord dl,9.78


In [ ]:
import numpy as np

# '?' 값을 NaN으로 대체
df['horsepower'].replace('?', np.nan, inplace=True)

# NaN이 있는 행 삭제
df.dropna(subset=['horsepower'], inplace=True)

# row 수 확인
df.shape[0]

392

In [ ]:
# horsepower column을 float으로 변환
df['horsepower'] = df['horsepower'].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    float64
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
 9   kpl           392 non-null    float64
dtypes: float64(6), int64(3), object(1)
memory usage: 33.7+ KB


In [ ]:
# origin column을 (1:USA, 2:EU, 3:Japan)로 변환
df['origin'] = df['origin'].replace([1,2,3], ['USA', 'EU', 'Japan'])
# df['origin'].replace({1:'USA', 2:'EU', 3:'Japan'}, inplace=True) # 위와 같음
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino,7.23


In [ ]:
# make origin column as categorical
df['origin'] = df['origin'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    int64   
 7   origin        392 non-null    category
 8   name          392 non-null    object  
 9   kpl           392 non-null    float64 
dtypes: category(1), float64(6), int64(2), object(1)
memory usage: 31.1+ KB


In [ ]:
# model year는 숫자로 의미가 없으므로 범주형으로 변환
df['model year'] = df['model year'].astype('category')

# model year sampling 수행 > 13개 카테고리 중 3개 뽑혔다 라고 나옴
df['model year'].sample(3)

25    70
59    72
93    73
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]

---
### 범주형(카테고리) 데이터 처리
- 연속 데이터를 그대로 사용하기 보다는 일정한 구간(binning)으로 나눠서 분석하는 것이 효율적인 경우가 많다

# 아래 구간 분할 많이많이 쓰임

In [ ]:
# horsepower의 np.histogram 확인
count, bin_dividers = np.histogram(df['horsepower'], bins=3) # histogram을 이용해서 구간을 3개로 나눔

In [ ]:
# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

df['hp_bin'] = pd.cut(df['horsepower'], bins=bin_dividers, labels=bin_names, include_lowest=True)

In [ ]:
df[['horsepower', 'hp_bin']].head(15)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


---
### 더미(dummy)변수
- 범주형 데이터를 회귀분석이나 머신러닝에 바로 사용할 수 없는 경우 사용
- 0 또는 1로 표현

In [ ]:
horsepower_dummies = pd.get_dummies(df['hp_bin'], prefix='hp')
df = pd.concat([df, horsepower_dummies], axis=1)
df.head(15)

# drop hp_저출력, hp_보통출력, hp_고출력 columns
df.drop(['hp_저출력', 'hp_보통출력', 'hp_고출력'], axis=1, inplace=True)

---
### One-Hot Encoding
- 범주형 데이터를 0, 1을 갖는 원핫벡터로 변환
- 결과는 선형대수학에서 정의하는 희소행렬(sparse matrix)로 변환한다.
- 희소행렬은 (행, 열) 좌표와 값 형태로 정리

In [ ]:
# import scikit-learn to one-hot encoding
from sklearn import preprocessing

# label_encoder, onehot_encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()


In [ ]:
# label_encoder로 문자열 범주를 숫자형으로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
onehot_labeled

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2])

In [ ]:
# 2차원 행렬로 형태 변경
onehot_reshaped = onehot_labeled.reshape(-1, 1)
onehot_reshaped

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [2]])

### 위에까지는 label_encoder, 여기부터는 onehot_encoder

In [ ]:
# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)

  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0


(0, 1) > 0번째 데이터는 1번째(보통출력)

(6, 0) > 6번째 데이터는 0번째(고출력)

(14, 2) > 14번째 데이터는 2번째(저출력) ...

---
### 정규화
- 각 열에 있는 숫자 데이터의 상대적 크기 때문에 머신러닝 분석결과가 달라질 수 있다.
- 숫자 데이터의 상대적인 크기 차이를 제거
- https://dine.tistory.com/77

In [ ]:
# horsepower col 형태 확인
df.horsepower.describe() # 46 ~ 230 사이 > 0~1사이로 몰자!

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64

In [ ]:
# normalize horsepower_1 = horsepower / abs(max(horsepower)) : 간단
horsepower_normalized_1 = df.horsepower / abs(df.horsepower.max())

horsepower_normalized_1.describe()

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64

In [ ]:
# horsepower_normalized_2 = 최대값과 최소값의 차
horsepower_normalized_2 = (df.horsepower - df.horsepower.min()) / (df.horsepower.max() - df.horsepower.min())

horsepower_normalized_2.describe()


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64

---
# 시계열 데이터
- 특정한 시점을 기록하는 Timestamp
- 두 지점 사이의 일정한 기간을 나타내는 Period

In [ ]:
df = pd.read_csv('data/stock-data.csv')
df.info() # date가 object 면 시계열 분석을 할 수 없다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


In [ ]:
df['new_Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB


In [ ]:
# make new_Date as index
df.set_index('new_Date', inplace=True)
df.head()

,Date,Close,Start,High,Low,Volume
new_Date,,,,,,
2018-07-02,2018-07-02,10100,10850,10900,10000,137977
2018-06-29,2018-06-29,10700,10550,10900,9990,170253
2018-06-28,2018-06-28,10400,10900,10950,10150,155769
2018-06-27,2018-06-27,10900,10800,11050,10500,133548
2018-06-26,2018-06-26,10800,10900,11000,10700,63039


In [ ]:
# drop Date column
df.drop('Date', axis=1, inplace=True)
df.head()

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


## Timestamp를 Period로 변환

In [ ]:
# 날짜 문자열 리스트
dates = ['2019-01-01', '2020-03-01', '2021-06-01']
ts_dates = pd.to_datetime(dates)
print(ts_dates)

pr_day = ts_dates.to_period(freq='D')
pr_month = ts_dates.to_period(freq='M')
pr_year = ts_dates.to_period(freq='A')
print(pr_day)
print(pr_month)
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)
PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


### PeriodIndex는 위처럼 Day, Month, Annual 등을 확인 가능
> freq Option   
> D (Day: 1일)   
> W (Week:1주)   
> M (Month end: 월말)   
> MS (Month begin: 월초)   
> Q (Quarter end : 분기말)   
> QS (Quarter begin: 분기초)  
> Q-APR (1, 4, 7, 10월만을 index로 사용)
> A (year end : 연말)   
> AS (year begin: 연초)   
> B (Business day : 휴일 제외)   
> H (hour : 1시간)   
> T (minute :1분)   
> S (second: 1초)

### 시계열 데이터 만들기

In [ ]:
# Timestamp의 배열 만들기 : 월 간격, 월의 시작일 기준
ts_ms = pd.date_range('2019-01-01', periods=6, freq='MS') # start, (end - 생략가능), periods, freq
print(ts_ms)

# ts_m: 월말기준, 2019 1월 ~ 10월, timezone 지정
ts_m = pd.date_range('2019-01-01', '2019-10-30', freq='M', tz='Asia/Seoul') # tz로 시간대 지정 가능
print(ts_m)

# ts_q: 분기기준, 2019 1월부터 8개 분기
ts_q = pd.date_range('2019-02-01', periods=8, freq='Q-APR') # 1 / 4 / 7 / 10월이 분기의 시작일
print(ts_q)

ts_q2 = pd.date_range('2019-02-01', periods=8, freq='Q-MAY') # 2 / 5 / 8 / 11월이 매 분기 시작점
print(ts_q2)

DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01'],
              dtype='datetime64[ns]', freq='MS')
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-08-31 00:00:00+09:00',
               '2019-09-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')
DatetimeIndex(['2019-04-30', '2019-07-31', '2019-10-31', '2020-01-31',
               '2020-04-30', '2020-07-31', '2020-10-31', '2021-01-31'],
              dtype='datetime64[ns]', freq='Q-APR')
DatetimeIndex(['2019-02-28', '2019-05-31', '2019-08-31', '2019-11-30',
               '2020-02-29', '2020-05-31', '2020-08-31', '2020-11-30'],
              dtype='datetime64[ns]', freq='Q-MAY')


---
# Period 배열

In [ ]:
# 1개월 길이
pr_m = pd.period_range('2022-01-01', periods=6, freq='M') # YYYY-MM 으로 월까지만 출력
print(pr_m)

# 1시간 길이
pr_h = pd.period_range('2022-01-01', periods=6, freq='H') # YYYY-MM-DD HH 으로 시간까지 출력
print(pr_h)

# 2시간 길이
pr_2h = pd.period_range('2022-01-01', periods=6, freq='2H') # YYYY-MM-DD HH:00:00 으로 2시간까지 출력
print(pr_2h)

PeriodIndex(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06'], dtype='period[M]')
PeriodIndex(['2022-01-01 00:00', '2022-01-01 01:00', '2022-01-01 02:00',
             '2022-01-01 03:00', '2022-01-01 04:00', '2022-01-01 05:00'],
            dtype='period[H]')
PeriodIndex(['2022-01-01 00:00', '2022-01-01 02:00', '2022-01-01 04:00',
             '2022-01-01 06:00', '2022-01-01 08:00', '2022-01-01 10:00'],
            dtype='period[2H]')


In [ ]:
# reset index
df.reset_index(inplace=True)
df.head()

,index,new_Date,Close,Start,High,Low,Volume
0,0,2018-07-02,10100,10850,10900,10000,137977
1,1,2018-06-29,10700,10550,10900,9990,170253
2,2,2018-06-28,10400,10900,10950,10150,155769
3,3,2018-06-27,10900,10800,11050,10500,133548
4,4,2018-06-26,10800,10900,11000,10700,63039


In [ ]:
df['Year'] = df.new_Date.dt.year
df['Month'] = df.new_Date.dt.month
df['Day'] = df.new_Date.dt.day
df.head()

,index,new_Date,Close,Start,High,Low,Volume,Year,Month,Day
0,0,2018-07-02,10100,10850,10900,10000,137977,2018,7,2
1,1,2018-06-29,10700,10550,10900,9990,170253,2018,6,29
2,2,2018-06-28,10400,10900,10950,10150,155769,2018,6,28
3,3,2018-06-27,10900,10800,11050,10500,133548,2018,6,27
4,4,2018-06-26,10800,10900,11000,10700,63039,2018,6,26


In [ ]:
df['Date-yr'] = df.new_Date.dt.to_period('A')
df['Date-m'] = df.new_Date.dt.to_period('M')
df['Date-d'] = df.new_Date.dt.to_period('D')
df.head()


,index,new_Date,Close,Start,High,Low,Volume,Year,Month,Day,Date-yr,Date-m,Date-d
0,0,2018-07-02,10100,10850,10900,10000,137977,2018,7,2,2018,2018-07,2018-07-02
1,1,2018-06-29,10700,10550,10900,9990,170253,2018,6,29,2018,2018-06,2018-06-29
2,2,2018-06-28,10400,10900,10950,10150,155769,2018,6,28,2018,2018-06,2018-06-28
3,3,2018-06-27,10900,10800,11050,10500,133548,2018,6,27,2018,2018-06,2018-06-27
4,4,2018-06-26,10800,10900,11000,10700,63039,2018,6,26,2018,2018-06,2018-06-26


In [ ]:
df.set_index('new_Date', inplace=True)
df.head()

KeyError: "None of ['new_Date'] are in the columns"

In [ ]:
# 날짜 인덱스를 이용해 데이터 선택하기
dy_y = df.loc['2018-07', 'Start': 'High']
dy_y.head()

,Start,High
time_delta,,
2018-07-02,10850,10900


In [ ]:
# 오늘로부터 180 ~ 190일 전 데이터 선택
today = pd.to_datetime('2018-12-25')
df['time_delta'] = today - df.index
df.set_index('time_delta', inplace=True)
df_180 = df['180 days', '189 days']
df_180

KeyError: ('180 days', '189 days')